# Using Google Geocoder

In [ ]:
from google_maps_geocoder.geocoder import GoogleGeocoder
import pandas as pd
import os

# Initialize the GoogleGeocoder
api_key = os.getenv("GOOGLE_API_KEY")
geocoder = GoogleGeocoder(api_key)

# Load the dataset
input_data = pd.read_csv(r"C:\Users\Miles.Vandenburg\Downloads\Meta_DataCenters.csv")

# Clean and prepare the data
destinations, needs_geocoding = geocoder.cleanup_pd(input_data)

# Perform geocoding
if needs_geocoding:
    final_dest_df = geocoder.geocode_addresses(destinations, needs_geocoding)
    final_dest_df.to_csv('geocoded_results.csv', index=False)
else:
    print("Data already contains coordinates.")

# tef = geocoder.geocode_single('1099 Stewart Street,Seattle,WA')
# # df = pd.json_normalize(tef)
# tef


# Using Signed Signature

In [1]:
# Import necessary function
from google_maps_geocoder.geocode_signed_url import signed_url_geocode
import os
from concurrent.futures import ThreadPoolExecutor


private_key = os.getenv("GOOGLE_MAPS_PRIVATE_KEY")
if not private_key:
    raise ValueError("GOOGLE_MAPS_PRIVATE_KEY is not set in the environment.")
# Define parameters
input_file = r"C:\Users\Miles.Vandenburg\Documents\C&W\Client_Work\McKesson\2025\0125-McKesson_DataScrub\Definitive_Healthcare\PhysicianGroupPracticeLocations_Active_0425_FilteredSpec_NotOwnedHospital.csv"
output_file = 'geocoded_results_signed_url.csv'
base_url = "http://maps.googleapis.com/maps/api/geocode/json"
client = "gme-cushmanwakefield"
channel = "GBC"
# Run the script
signed_url_geocode(input_file, output_file, private_key, base_url, client, channel, geocode_only = True)


Signed URLs have been read from results\signed_urls.csv
Processing 105315 URLs with parallel processing...
Processed 100/105315 URLs
Processed 200/105315 URLs
Processed 300/105315 URLs
Processed 400/105315 URLs
Processed 500/105315 URLs
Processed 600/105315 URLs
Processed 700/105315 URLs
Processed 800/105315 URLs
Processed 900/105315 URLs
Processed 1000/105315 URLs
Processed 1100/105315 URLs
Processed 1200/105315 URLs
Processed 1300/105315 URLs
Processed 1400/105315 URLs
Processed 1500/105315 URLs
Processed 1600/105315 URLs
Processed 1700/105315 URLs
Processed 1800/105315 URLs
Processed 1900/105315 URLs
Processed 2000/105315 URLs
Processed 2100/105315 URLs
Processed 2200/105315 URLs
Processed 2300/105315 URLs
Processed 2400/105315 URLs
Processed 2500/105315 URLs
Processed 2600/105315 URLs
Processed 2700/105315 URLs
Processed 2800/105315 URLs
Processed 2900/105315 URLs
Processed 3000/105315 URLs
Processed 3100/105315 URLs
Processed 3200/105315 URLs
Processed 3300/105315 URLs
Processed 3

,formatted_address,latitude,longitude,accuracy,google_place_id,type,postcode,number_of_results,status,Site_Number,...,ACO Affiliations,HIE Affiliations,Payor Affiliations,GPO Affiliations,Network Name,Company Status,Network Parent Name,Management Service Organization,Definitive Network Parent ID,ADDRESS_FULL
0,"22030 Sherman Way #101, Canoga Park, CA 91303,...",34.200827,-118.607197,ROOFTOP,ChIJiR24aD-cwoARMjtd-C1mlh8,"street_address,subpremise",91303,1,OK,1,...,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN,NaN,"22030 Sherman Way,Ste 101,Canoga Park,CA,91303..."
1,"6960 108th St Ste 108, Forest Hills, NY 11375,...",40.723282,-73.844833,ROOFTOP,ChIJOS1XrSdewokRISTQr6cvCoE,"street_address,subpremise",11375,1,OK,2,...,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN,NaN,"69-60 108th St,Ste 108,Forest Hills,NY,11375,3..."
2,"5450 Clearfork Main St Suite 460, Fort Worth, ...",32.708069,-97.401554,ROOFTOP,ChIJU2I2-cJzToYRpGeYMuruTRk,"street_address,subpremise",76109,1,OK,3,...,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN,NaN,"5450 Clearfork Main St,Ste 460,Fort Worth,TX,7..."
3,"3587 12th St, Wayland, MI 49348, USA",42.679993,-85.661875,ROOFTOP,ChIJWReX0EDMGYgR4o9IA5qewwM,"premise,street_address",49348,1,OK,4,...,West Michigan Accountable Care Organization LLC,NaN,NaN,NaN,NaN,Active,NaN,NaN,NaN,"3587 12th St,Wayland,MI,49348,466"
4,"3418 Broadway # 2, New York, NY 10031, USA",40.822135,-73.953210,ROOFTOP,ChIJO8dX0WX2wokRcbIOMPoCG5Y,"street_address,subpremise",10031,1,OK,5,...,Somos ACO,NaN,NaN,NaN,NaN,Active,NaN,NaN,NaN,"3418 Broadway,Ste 2,New York,NY,10031,3,569"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105310,"1728 W Glendale Ave #303, Phoenix, AZ 85021, USA",33.538960,-112.096778,ROOFTOP,ChIJ0Wsvk9hsK4cRFv3-72tzAts,"street_address,subpremise",85021,1,OK,105311,...,NaN,NaN,NaN,NaN,NaN,Active,NaN,NaN,NaN,"1728 W Glendale Ave,Ste 303,Phoenix,AZ,85021,411"
105311,"1050 Kings Hwy #112, Cherry Hill Township, NJ ...",39.904652,-75.021750,ROOFTOP,EjgxMDUwIEtpbmdzIEh3eSAjMTEyLCBDaGVycnkgSGlsbC...,"street_address,subpremise",08034,1,OK,105312,...,Community Care Collaborative of Pennsylvania a...,NaN,NaN,HealthTrust Performance Group (FKA HealthTrust...,NaN,Active,NaN,NaN,NaN,"1050 Kings Hwy,Ste 112,Cherry Hill,NJ,8034,530"
105312,"6417 Broadway, Flushing, NY 11377, USA",40.749623,-73.898029,ROOFTOP,ChIJEzzh9gRfwokRhqGo2_34PBE,"street_address,subpremise",11377,1,OK,105313,...,Somos ACO,NaN,NaN,NaN,NaN,Active,NaN,NaN,NaN,"6417 Broadway,Woodside,NY,11377"
105313,"171-09 Mayfield Rd, Jamaica, NY 11432, USA",40.714451,-73.792057,ROOFTOP,ChIJtSnytB1hwokR4LHP0E-pOYc,"street_address,subpremise",11432,1,OK,105314,...,Somos ACO,NaN,NaN,NaN,NaN,Active,NaN,NaN,NaN,"17109 Mayfield Rd,Jamaica,NY,11432"
